In [1]:
import pickle
from collections import defaultdict

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.cluster import AffinityPropagation

# Load

In [2]:
ROME_df = pd.read_csv('../referentiels/referentiel_ROME/20150921_arboprincipale28427_ROME.csv', index_col=0, sep='|', dtype=str)
OGR_df = pd.read_csv('../referentiels/referentiel_OGR/20150921_arboprincipale28427_OGR.csv', sep='|', dtype=str).set_index('OGR')
NAF_df = pd.read_csv('../referentiels/referentiel_NAF/naf2008_liste_n5_nouveau_header.csv', sep='|', encoding="utf-8").set_index(['NAF'])

In [3]:
with open('array_offres_OGR.pickle', 'rb') as f:
    offres_OGR = pickle.load(f)

# Data transformation

In [4]:
def get_matrix(ROME):
    ROME_OGRs = list(OGR_df[OGR_df.ROME == ROME].index)
    ROME_OGR_indexes = [OGR_df.index.get_loc(OGR) for OGR in ROME_OGRs]
    
    ROME_matrix = offres_OGR[:, ROME_OGR_indexes]
    
    is_positive = ROME_matrix.sum(0)>0
    is_positive_mask = np.where(is_positive)[0]
    ROME_matrix = ROME_matrix[:, is_positive_mask]
    ROME_OGRs = [ROME_OGRs[i] for i in is_positive_mask]
    
    return ROME_matrix, ROME_OGRs

In [ ]:
list(enumerate([OGR_df.loc[OGR].label for OGR in ROME_OGRs]))

In [5]:
def similarity(A):
    norm2 = np.power(A, 2).sum(0)
    norm = np.sqrt(norm2)
    A_normalized = A/norm
    product = np.dot(A_normalized.T, A_normalized)
    return product

In [ ]:
plt.imshow(sim, interpolation='none')

In [9]:
def print_clusters(clusters, ROME_OGRs):
    for cl in clusters:
        for el in cl:
            OGR = ROME_OGRs[el]
            print(OGR_df.loc[OGR].label)
        print()

# Naive clustering

In [ ]:
bool_sim = sim>0.7
equi = np.linalg.matrix_power(bool_sim, 100)
plt.imshow(equi, interpolation='none')

In [ ]:
clusters = defaultdict(list)
for i in range(len(ROME_OGRs)):
    repres = min(np.where(equi[i, :])[0])
    clusters[repres].append(i)
clusters = list(clusters.values())
clusters

In [ ]:
print_clusters(clusters)

# Affinity propagation

In [30]:
def compute_clusters(sim):
    if len(sim) == 0:
        return []
    if len(sim) == 1:
        return [[0]]
    
    model = AffinityPropagation(affinity='precomputed')
    labels = model.fit_predict(sim)
    clusters = defaultdict(list)
    for i, l in enumerate(labels):
        clusters[l].append(i)
    clusters = list(clusters.values())
    return clusters

# Loop

In [31]:
def print_ROME(ROME):
    ROME_label = ROME_df.loc[ROME].label
    print('*** {} : {} ***'.format(ROME, ROME_label))
    print()
    ROME_matrix, ROME_OGRs = get_matrix(ROME)
    sim = similarity(ROME_matrix)
    clusters = compute_clusters(sim)
    print_clusters(clusters, ROME_OGRs)
    print()

In [32]:
print_ROME('D1106')

*** D1106 : Vente en alimentation ***

Epicier / Epicière
Vendeur / Vendeuse en boulangerie-pâtisserie

Marchand / Marchande de fruits et légumes
Vendeur / Vendeuse de fruits et légumes
Vendeur / Vendeuse de primeurs

Vendeur / Vendeuse en boucherie
Vendeur / Vendeuse en boucherie-charcuterie
Vendeur / Vendeuse en charcuterie
Vendeur / Vendeuse en poissonnerie
Vendeur / Vendeuse en produits frais
Vendeur / Vendeuse en rôtisserie
Vendeur / Vendeuse rayon traiteur

Vendeur / Vendeuse en chocolaterie
Vendeur / Vendeuse en confiserie

Vendeur / Vendeuse en alimentation générale
Vendeur / Vendeuse en crèmerie
Vendeur / Vendeuse en épicerie
Vendeur / Vendeuse en épicerie fine
Vendeur / Vendeuse en produits biologiques
Vendeur / Vendeuse en produits diététiques
Vendeur / Vendeuse rayon fromages

Vendeur / Vendeuse en vins et spiritueux




In [34]:
for ROME in ROME_df.index:
    print_ROME(ROME)

*** A1101 : Conduite d'engins agricoles et forestiers ***

Conducteur / Conductrice de matériels de semis

Conducteur / Conductrice de pulvérisateur

Conducteur / Conductrice d'automoteur de récolte
Conducteur / Conductrice de tête d'abattage
Conducteur / Conductrice de tracto-benne
Conducteur / Conductrice d'engins de débardage
Conducteur / Conductrice d'épareuse
Opérateur / Opératrice d'épandage

Chauffeur / Chauffeuse de machines agricoles
Conducteur / Conductrice d'engins d'exploitation agricole
Opérateur / Opératrice d'abatteuse

Conducteur / Conductrice d'abatteuses
Conducteur / Conductrice d'engins d'exploitation forestière
Conducteur / Conductrice d'engins forestiers
Débardeur / Débardeuse
Débardeur forestier / Débardeuse forestière
Pilote de machines d'abattage
Tractoriste forestier / Tractoriste forestière

Conducteur / Conductrice de machines à vendanger
Conducteur / Conductrice de tracteur
Tractoriste agricole


*** A1201 : Bûcheronnage et élagage ***

Arboriste
Bûcheron / 

In [33]:
print_ROME('J1304')


*** J1304 : Aide en puériculture ***

Auxiliaire de puériculture


